# Captura y almacenamiento de la información
Profesores: MSc Oscar Bría - Mg Javier Bazzocco


## Introducción y objetivos

Twitter es una de las mayores redes sociales en cuanto al flujo de información. Dada su gran popularidad entre personas de distintas ubicaciones geográficas, edades, aspectos sociales, etc...en esta red puede encontrarse información de las mas diversas areas (política, deportes, ciencia, chismes, etc...). Si bien un tweet puede tener un número bajo de caracteres, la información suele estar bien resumida y esto lo hace una herramienta ágil para la búsqueda inmediata de información sobre algún tema particular. Por ejemplo, basta buscar con un hashtag (#palabraclave) sobre un tema de interés para que tengamos la información mas actualizada sobre esa palabra clave. Hoy en día no solo las personas utilizan estas redes sociales para transmitir información, sino que también lo hacen las empresas, instituciones educativas y científicas, gobiernos, etc...



En este contexto, el objetivo de esta notebook es presentar una forma de obtener información de dicha red social a partir de 
una palabra clave y almacenar dichos datos en una base de datos como MongoDB o un archivo CSV.

### Comenzemos...

Para llevar a cabo el objetivo planteado se utilizará el lenguaje de programación Python y la librería Tweepy para interactuar con la API de Twitter. Una vez obtenidos los datos serán almacenados en MongoDB o un archivo CSV. Para interactuar con mongo mediante Python se puede utilizar pymongo. En este trabajo se utiliza MongoDB Atlas, esto es la base de datos como servicio de MongoDB (DaS) el cual permite tener una cuenta gratuita con un espacio limitado pero suficiente para este tipo de análisis. https://www.mongodb.com/cloud/atlas/signup?v=1

El primer paso es importar las librerías de Python (para instalar por primera vez los módulos descomentar las lineas con !): 

In [1]:
# Se importan las librerías. Para instalar tweepy, pymongo o dnspython descomentar las lineas. 
import tweepy as tw # para interactuar con la API de Twitter
#!pip install tweepy 
import pandas as pd # para manipular datos
import csv          # archivos comma separation value
import json         # archivos JSON
import pprint
#!pip install dnspython 
#!pip install pymongo     
import numpy as np


Para conectarse a Mongo se necesita cierta info que se obtiene desde la cuenta. Luego se puede entrar a la base de datos generada en el cluster de MongoDB Atlas:

In [1]:
from pymongo import MongoClient
client = MongoClient(" credenciales ")
db = client.get_database('Prueba0') # Entro en la base de datos Prueba0 que generé en mi cluster en MongoDB Atlas.

Una vista del entordo de Mongo

<img src="fig_mongo.png">

Desde la notebook puede verse las colecciones ya creadas:

In [3]:
collects = db.collection_names(include_system_collections=False)
collects

<ipython-input-3-f201acba4b4f>:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  collects = db.collection_names(include_system_collections=False)


['Sismos', 'Tweets']

Se elije en donde guardar la información

In [4]:
collection = db['Tweets'] 

Ahora, para poder acceder a la información de Twitter hay que tener una cuenta en twitter developers (https://developer.twitter.com/en). Ahí se crea una aplicación, y luego de un par de preguntas sobre el objetivo de la App... listo! Se necesitan 4 claves de la aplicacion de twitter. Se puede copiar directamente desde la cuenta de twitter devs.

In [2]:
consumer_key= ''
consumer_secret= ''
access_token= ''
access_token_secret= ''

Luego, para crear la conexión...

In [6]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Una vez que está todo preparado para interactuar con Twitter se puede comenzar a trabajar en la colección de tweets. Para poder recolectar los posteos sobre un tema en particular se puede filtrar por ubicación. De esta forma es necesario contar con las coordenadas geográficas y un radio (en kilometros) de interés. Las coordenadas a continuación son para Capital Federal y alrededores. 



In [7]:
latitud = -34.6131516    
longitud = -58.3772316
radio = 50             # Rango de búsqueda en km en torno a la posición dada.

Una limitación que tiene esta forma de interactuar con Twitter es que se pueden obtener los tweets de los últimos 7 días. Otros parámetros que hay que seleccionar son la cantidad de tweets que se quieren capturar, la palabra clave sobre el tema que nos interese, y es posible determinar (dentro de la semana de gracia) la fecha de los tweets.

En este ejemplo buscaremos tweets acerca del Bitcoin que esta semana rompió la barrera de los 40 mil $

In [8]:
search_words = "btc" # se elige la palabra clave de interés
#search_words_f = search_words + "-filter:retweets" # se filtran los retweets
max_res = 10            # numero maximo de resultados que quiero ver.
date_since = "2021-1-2" # la fecha (anio/mes/dia) desde...
date_till =  "2021-1-8" # hasta...

# Extraemos los tweets...
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="es",
              geocode = "%f,%f,%dkm" % (latitud, longitud, radio),
              since=date_since,
              until=date_till).items(max_res)

Luego, se extrae el usuario, ubicación (ciudad,provincia, país) y tweet y se lo guarda en un tipo de dato llamado dataframe:

In [9]:
users_locs = [[tweet1.user.screen_name, tweet1.user.location, tweet1.text] for tweet1 in tweets] 
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['usuario', 'ubicacion','Tweets']) # Se convierte la info capturada a un dataframe
fil,col = tweet_text.shape # nos quedamos con las dimensiones del dataframe
tweet_text

,usuario,ubicacion,Tweets
0,Diegobarthalot1,"Buenos Aires, Argentina",@arielsbdar Siempre que BTC siga subiendo o la...
1,ianaya89,Buenos Aires,"@ddgn No esta mal usarlos para empezar, porque..."
2,FSives,"La Plata, Argentina","Como referencia, sólo el sitio freebitco tiene..."
3,ianaya89,Buenos Aires,@ddgn Que tecnicamente no sos dueño de tus BTC...
4,FSives,"La Plata, Argentina","Primero. ¿Son monedas? Esto es, un elemento de..."
5,FSives,"La Plata, Argentina",Criptomonedas. ¿La nueva fase del capitalismo?...
6,nigromantevader,Buenos aires. Argentina,"@rjschlindwein El btc, puede bajar? Obvio, igu..."
7,Mantrellbolsero,La Plata,@verajuanp Ojo q me pasó lo mismo cuando el BT...
8,EmilioOlveira,Buenos Aires - Argentina,$btc $eth $xrp ... otro derrape fuerte.
9,santihelou,"Buenos Aires, Argentina",@elrama_ Un poco forzado el ejemplo de Argenti...


Esta estructura permite guardar la información obtenida facilmente en archivos CSV. Para poder guardarla en Mongo hay un paso intermedio donde se lleva la información de una lista a un diccionario. Sin embargo, para poder cargarlo en Mongo es necesario 
trabajar un poco con las claves llevándolas a tipo string.

In [27]:
u = np.arange(0,fil)
k = [] # lista donde van a estar las claves del diccionario

for i in u:
    k.append(str(u[i]))
    
d = dict(zip(v,users_locs)) 

Una vez que ya tenemos el diccionario con los tweets podemos guardarlos en la base de datos o en un archivo csv. Para guardar en Mongo solo hay que cambiar los valores donde dice opción.

In [29]:
opcion = 2 # dejar uno para guardar en csv o poner cualquier otro número para guardar en la BD.
outfile = 'salida'+ search_words +'.csv'
if op == 1:
    tweet_text.to_csv(outfile,index=False,sep=',')
else:
    collection.insert_one(d)


En la Figura puede verse los tweets guardados:

<img src="fig_tweets.png">